### Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os, sys
from src.model import MFMViT
from src.fft_utils import *
from src.loss import MFMLoss
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR, LambdaLR

c:\Users\admin\miniconda3\envs\deepfake-detection\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not load this library: C:\Users\admin\miniconda3\envs\deepfake-detection\Lib\site-packages\torchvision\image.pyd'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### Trainer

In [3]:
def trainer():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    img_size = 224
    batch_size = 16
    lr = 1e-4
    epochs = 50
    mask_ratio = 0.5
    warmup_epochs = 5
    
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    train_dataset = datasets.ImageFolder('E:/data/train', transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    val_dataset = datasets.ImageFolder('E:/data/val', transform=transform) if os.path.exists('E:/data/val') else None
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) if val_dataset else None
    
    model = MFMViT(img_size=img_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = MFMLoss().to(device)
    
    warmup_scheduler = LambdaLR(optimizer, lambda epoch: min(1.0, (epoch + 1) / warmup_epochs))
    cosine_scheduler = CosineAnnealingLR(optimizer, T_max=epochs - warmup_epochs, eta_min=1e-6)
    
    os.makedirs('checkpoints', exist_ok=True)
    
    for epoch in range(epochs):
        model.train()
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
        train_loss = 0
        
        for images, _ in pbar:
            images = images.to(device)
            
            fft_original = apply_fft(images)
            amplitude_original = get_spectrum_amplitude(fft_original)
            
            mask = get_mask(images.shape[0], images.shape[1], img_size, 
                          ratio=mask_ratio, device=device)
            fft_masked = fft_original * mask
            
            corrupted_spatial = apply_ifft(fft_masked)
            predicted_spatial = model(corrupted_spatial)
            
            fft_predicted = apply_fft(predicted_spatial)
            amplitude_predicted = get_spectrum_amplitude(fft_predicted)
            
            loss = criterion(amplitude_predicted, amplitude_original, mask)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        val_loss = 0
        if val_loader:
            model.eval()
            with torch.no_grad():
                for images, _ in val_loader:
                    images = images.to(device)
                    
                    fft_original = apply_fft(images)
                    amplitude_original = get_spectrum_amplitude(fft_original)
                    
                    mask = get_mask(images.shape[0], images.shape[1], img_size, 
                                  ratio=mask_ratio, device=device)
                    fft_masked = fft_original * mask
                    
                    corrupted_spatial = apply_ifft(fft_masked)
                    predicted_spatial = model(corrupted_spatial)
                    
                    fft_predicted = apply_fft(predicted_spatial)
                    amplitude_predicted = get_spectrum_amplitude(fft_predicted)
                    
                    loss = criterion(amplitude_predicted, amplitude_original, mask)
                    val_loss += loss.item()
            
            val_loss /= len(val_loader)
        
        if epoch < warmup_epochs:
            warmup_scheduler.step()
        else:
            cosine_scheduler.step()
        
        train_loss /= len(train_loader)
        msg = f'Epoch {epoch+1} | Train Loss: {train_loss:.4f}'
        if val_loader:
            msg += f' | Val Loss: {val_loss:.4f}'
        msg += f' | LR: {optimizer.param_groups[0]["lr"]:.2e}'
        print(msg + '\n')
        
        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), f'checkpoints/mfm_vit_epoch_{epoch+1}.pth')
    
    torch.save(model.state_dict(), 'checkpoints/mfm_vit_final.pth')
    print('Training complete!')

trainer()

Epoch 1/50:   0%|          | 0/5014 [00:00<?, ?it/s]

[MFMViT init] embed_dim=768, num_patches=196, patches_per_side=14, patch_size=16, patch_dim=768


Epoch 1/50: 100%|██████████| 5014/5014 [14:01<00:00,  5.96it/s, loss=0.2811]


Epoch 1 | Train Loss: 0.3265 | Val Loss: 0.3008 | LR: 4.00e-05



Epoch 2/50: 100%|██████████| 5014/5014 [13:52<00:00,  6.02it/s, loss=0.2861]


...

Epoch 48 | Train Loss: 0.1613 | Val Loss: 0.1611 | LR: 1.48e-06



Epoch 49/50: 100%|██████████| 5014/5014 [13:43<00:00,  6.09it/s, loss=0.1772]


Epoch 49 | Train Loss: 0.1611 | Val Loss: 0.1610 | LR: 1.12e-06



Epoch 50/50: 100%|██████████| 5014/5014 [13:41<00:00,  6.11it/s, loss=0.1574]


Epoch 50 | Train Loss: 0.1609 | Val Loss: 0.1609 | LR: 1.00e-06

Training complete!
